In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn.metrics import make_scorer
from sklearn.model_selection import ShuffleSplit
from scipy import stats
import matplotlib.pyplot as plt
from matplotlib import pylab
import random
from sklearn.model_selection import KFold
import plotly.express as px
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from mlxtend.plotting import plot_sequential_feature_selection as plot_sfs
from sklearn.ensemble import RandomForestRegressor
import jupyternotify
ip = get_ipython()
ip.register_magics(jupyternotify.JupyterNotifyMagics)
from  sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import r2_score
from sklearn.svm import SVR
import dateutil
from datetime import datetime
from dateutil.relativedelta import relativedelta
from mlxtend.evaluate import PredefinedHoldoutSplit
import kneed
import kaleido
%run functions.ipynb

In [ ]:
station = "kahl"

In [ ]:
#OPTIONS:  "OPO4P", "NO3N", "NH4N", "TRP"
predict = "NO3N"
test_model = "rf"

In [ ]:
if test_model == "rf":
    algo = RandomForestRegressor(n_jobs = -1)
elif test_model == "lr":
    algo = linear_model.LinearRegression()

In [ ]:
# Here we are inputting the cleaned data that has been saved
df = pd.read_csv('%s_%s_final.csv' %(station, predict))
df['Datum'] = pd.to_datetime(df['Datum'])
df= df.set_index("Datum")
df

In [ ]:
df.describe()

In [ ]:
# this cell splits the data into 80:20 ratio, for training and testing. Training dataset is named manual_train
# testing dataset is named manual_test

list_feat = df.columns.drop(predict)
ss = df.reset_index()
    
column_headers = [x for x in ss.columns]
time_df = pd.DataFrame(columns= column_headers)

last_date = ss.iloc[-2,0]
temp_date = ss.iloc[0,0]
i = 0
while temp_date.month != last_date.month or temp_date.year != last_date.year:
    temp_date = ss.iloc[0,0] + pd.DateOffset(months =i)
    temp = ss[(ss.Datum.dt.month == temp_date.month) & (ss.Datum.dt.year==temp_date.year) ]
                
    temp2 = temp.sample(frac = 20*0.01, random_state = 1)
            
    time_df = pd.concat([time_df, temp2], axis = 0)
    i = i+1


time_df = time_df.drop_duplicates(subset=['Datum'], keep='first')
dates = np.array(time_df["Datum"])
time_df= time_df.set_index("Datum")
manual_train = ss.set_index("Datum")
manual_train.drop(dates, inplace=True)

manual_test = time_df

# here both the test and train dataframe are included with timestamps

In [ ]:
# This is to load a predefined manual_test value, so that it's not random everytime
manual_test = pd.read_csv('%s_%s_test.csv' %(station, predict))
manual_test['Datum'] = pd.to_datetime(manual_test['Datum'])
manual_test= manual_test.set_index("Datum")
manual_test

In [ ]:
manual_test.describe()

### change tt to manual_train

In [ ]:
if test_model == "lr" and station == "kahl" and predict == "NO3N":
    list_feat = ["month_sin","Temp","pH","week_sin","Conduct"]
    
if test_model == "lr" and station == "kahl" and predict == "OPO4P":
    list_feat = ["O2","pH","month_sin", "month_cos", "week_sin"]

if test_model == "lr" and station == "kahl" and predict == "NH4N":
    list_feat = ["week_cos","Conduct","pH", "week_sin","month_sin"]
    
if test_model == "lr" and station == "erlabrunn" and predict == "NO3N":
    list_feat = ["month_sin","Temp", "Conduct","week_sin","month_cos"]

if test_model == "lr" and station == "erlabrunn" and predict == "OPO4P":
    list_feat = ["month_sin","flow","O2","Temp","Conduct"]
    

In [ ]:
if test_model == "rf" and station == "kahl" and predict == "NO3N":
    list_feat = ["week_sin","Conduct","Temp","O2","month_sin"]  

if test_model == "rf" and station == "kahl" and predict == "OPO4P":
    list_feat = ["week_sin","Conduct","Temp","flow","O2"]
    
if test_model == "rf" and station == "kahl" and predict == "NH4N":
    list_feat = ["week_sin","Conduct","Temp","O2","flow"]
    
if test_model == "rf" and station == "erlabrunn" and predict == "NO3N":
    list_feat = ["week_sin","Conduct","Temp","flow","pH"]

if test_model == "rf" and station == "erlabrunn" and predict == "OPO4P":
    list_feat = ["week_sin","flow","Conduct","Temp","O2"]

### Grid Search

In [ ]:
# Number of trees in random forest
n_estimators = [50]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [10, 20, 30]
# Minimum number of samples required to split a node
min_samples_split = [6, 12, 20]
# Minimum number of samples required at each leaf node
min_samples_leaf = [6, 12, 20]


# Create the param grid
param_grid = {'n_estimators': n_estimators,
              'max_features': max_features,
              'max_depth': max_depth,
              'min_samples_split': min_samples_split,
              'min_samples_leaf': min_samples_leaf,
              'bootstrap': bootstrap}
print(param_grid)
from sklearn.model_selection import GridSearchCV
rf_Grid = GridSearchCV(estimator = algo, param_grid = param_grid, cv = 5, verbose=2, n_jobs = -1)

In [ ]:
X = np.array(manual_train[list_feat])
y = np.array(manual_train[[predict]])

rf_Grid.fit(X, y.ravel())

In [ ]:
best_hyper = rf_Grid.best_params_
print(best_hyper)

fitted_rf = RandomForestRegressor(bootstrap = best_hyper.get('bootstrap'), 
                                  max_depth= best_hyper.get('max_depth'),
                                  max_features= best_hyper.get("max_features"),
                                  min_samples_leaf= best_hyper.get("min_samples_leaf"),
                                  min_samples_split = best_hyper.get("min_samples_split") ,
                                  n_estimators=100, n_jobs = -1)

### Testing on the unseen dataset i.e the 20% of the earlier separated dataset. The model is trained on the top 5 features that are mentioned above. 

In [ ]:
ss = manual_train
tt = manual_test

arr_r2 = []
arr_RMSE = []

# list_feat = manual_train.columns.drop(predict)

    #train
X_train = np.array(manual_train[list_feat])
y_train = np.array(manual_train[[predict]])

    #test
X_test = np.array(manual_test[list_feat])
y_test = np.array(manual_test[[predict]])

model = fitted_rf
model.fit(X_train,  y_train.ravel())
predictions = model.predict(X_test)

temp_r2 = r2_score(y_test, predictions)
arr_r2.append(temp_r2)

temp_rmse = mean_squared_error(y_test, predictions, squared=False)
arr_RMSE.append(temp_rmse)

print("Rsquared", predict, "\n", temp_r2)
print('RMSE ', predict, '\n', temp_rmse)
print( "***")

In [ ]:
# Plotting the oberved vs predicted curve for the run on testing dataset one cell above

rr = pd.DataFrame(predictions, columns = {"Predicted_%s" %(predict)}).round(2)
rr =  pd.concat([manual_test[[predict]].reset_index(),rr], axis =1 ).sort_values('Datum').set_index("Datum")


figx = px.line(rr, x = rr.index, y=[predict, "Predicted_%s" %(predict)], title = "%s" %(station))
figx.update_xaxes(rangeslider_visible = True)
# figx.update_layout(yaxis_range=[0, 0.2])
print("full")
print("Rsquared", predict, "\n", temp_r2)
print('RMSE ', predict, '\n', temp_rmse)
figx.show()

### Determination of Knee

In [ ]:
arr_r2 = []
arr_RMSE = []
samples_per_month = [1, 3, 5, 10, 20, 30, 80, 100,120, 150, 175, 180, 190, 200, 210, 225, 250, 275, 300, 325, 350,
                     375, 400, 425, 450, 475, 500,600,700, 800, 1000, 1100, 1200, 1300, 1400, 1500, 1600, 1700,
                     1800, 2001, 2200, 2400]
# samples_per_month = [samples]

In [ ]:
# Training the vanilla RF model on 40 different sampling frequencies and testing them on the 20% of the entire dataset.
for j in samples_per_month:
    ss = manual_train.reset_index()
    tt = manual_test
    
    boss_list = [x for x in ss.columns]
    time_df = pd.DataFrame(columns= boss_list)

    last_date = ss.iloc[-2,0]
    temp_date = ss.iloc[0,0]
    i = 0
    while temp_date.month != last_date.month or temp_date.year != last_date.year:
        temp_date = ss.iloc[0,0] + pd.DateOffset(months =i)
        temp = ss[(ss.Datum.dt.month == temp_date.month) & (ss.Datum.dt.year==temp_date.year) ]
#         print("Number of data points in ", temp_date.month, " : ", len(temp) )
        
        if len(temp) > j:
            temp2 = temp.sample(n = j, random_state = 1)
        elif len(temp) < j:
            temp2 = temp.sample(frac = 0.95, random_state = 1)
            
        time_df = pd.concat([time_df, temp2], axis = 0)
        i = i+1

    time_df = time_df.drop_duplicates(subset=['Datum'], keep='first')
#     dates = np.array(time_df["Datum"])
    time_df = time_df.set_index("Datum")
#     ss = ss.set_index("Datum")
#     ss.drop(dates, inplace=True)
    
#     display(ss)
#     display(time_df)

#     list_feat = time_df.columns.drop(predict)   NOT USING THIS JUST TO SEE HOW MUCH IT CHANGES IF WE USE LESSER PREDICTORS

    #train
    X_train = np.array(time_df[list_feat])
    y_train = np.array(time_df[[predict]])

    #test
    X_test = np.array(tt[list_feat])
    y_test = np.array(tt[[predict]])

    model = RandomForestRegressor(n_jobs = -1)
    model.fit(X_train,  y_train.ravel())
    predictions = model.predict(X_test)

    temp_r2 = r2_score(y_test, predictions)
    arr_r2.append(temp_r2)

    temp_rmse = mean_squared_error(y_test, predictions, squared=False)
    arr_RMSE.append(temp_rmse)

    print(j)
    print("Rsquared", predict, "\n", temp_r2)
    print('RMSE ', predict, '\n', temp_rmse)
    print( "***")

In [ ]:
# Combining the result of 40 runs in a pandas dataframe
results = pd.DataFrame(list(zip(samples_per_month, arr_r2, arr_RMSE)),
               columns =['Samples Per Month', 'R2', 'RMSE'])
results = results.sort_values('Samples Per Month')
results.to_csv(r'%s_%s_numbers_3_more_freq.csv' %(station, predict), index = False, header = True)
results

In [ ]:
# Determination of knee using kneedle algorithm

x = np.array(results[["Samples Per Month"]].div(2592000)).squeeze()
y = np.array(results[["RMSE"]]).squeeze()

kneedle = kneed.KneeLocator(x, y, S=9.0, curve="convex", direction="decreasing")
freq = round(kneedle.knee, 10)  # knee point frequency in Hz

print(station, predict)
print ("Frequency: ", freq)

freq_m = freq*30*24*60*60   # Converting the frequency in Hz to samples per month
print("Samples per month: ", freq_m)   

print("Hourly frequency: ", round(30/freq_m*24, 10), " hours") # Converting the frequency in Hz to hourly

kneedle.plot_knee()

In [ ]:
tt = tt

rr = pd.DataFrame(predictions, columns = {"Predicted_%s" %(predict)}).round(2)
rr =  pd.concat([tt[[predict]].reset_index(),rr], axis =1 ).sort_values('Datum').set_index("Datum")

figx = px.line(rr, x = rr.index, y=[predict, "Predicted_%s" %(predict)], title = "%s " %(station))
figx.update_xaxes(rangeslider_visible = True)
# figx.update_layout(yaxis_range=[0, 0.2])

print(samples_per_month)      #no use, it's here just for marking
print("Rsquared", predict, "\n", temp_r2)
print('RMSE ', predict, '\n', temp_rmse)

figx.show()